In [ ]:
library(tidyverse) # metapackage with lots of helpful functions
library(dplyr)
library(ggplot2)
install.packages("ggpubr")
library(ggpubr)
install.packages("rworldmap")
library(rworldmap)

In [ ]:
df <- read.csv(file="../input/gtd/globalterrorismdb_0718dist.csv", header=TRUE, sep=",")

In [ ]:
summary(df)

In [ ]:
weaponcnt<-df%>%group_by(success,weaptype1_txt)%>%summarise(count=n())
weaponcnt

In [ ]:
weapprop <- mutate(weaponcnt, prop = count/sum(weaponcnt[which(weaponcnt$success==success,),"count"]))
#weapprop

In [ ]:
dim(df[which(df$success==1,),])
dim(df[which(df$success==0,),])

**The below plot indicates that in all the cases of failure there are only few attacks with firearms hence low proportion similarly with explosive**

In [ ]:
#weaponcnt
theme_set(theme_pubr())

ggplot(weapprop, aes(x=success, y=prop,group=weaptype1_txt,color=weaptype1_txt)) +
  geom_bar(stat = "identity",position = "dodge")+xlab("success at 1 and faiure at 0")+ylab("proportion of success or failure in each category")

In [ ]:
x=ggsave(file="bench_query_sort.pdf", width=4, height=4, dpi=300)

In [ ]:
sui<-df%>%group_by(success,suicide)%>%summarise(count=n())
sui

In [ ]:
sui1=mutate(sui, prop = count/sum(sui[which(sui$success==success,),"count"]))
sui2=sui1[which(sui1$suicide==1,),]
sui2

In [ ]:
ggplot(sui2, aes(x=success, y=count)) +
  geom_bar(stat = "identity")+xlab("success =1 and failure=0 in all suicide attack-20% of suicide attacks have failed ")+ylab("count")

In [ ]:
ggplot(df, aes(iyear)) +
  geom_bar(fill = "#0073C2FF")+ggtitle("trend of no. of attacks over year")+theme(axis.text.x = element_text(angle = 90, hjust =1, vjust = 0.2))+scale_x_continuous("YEAR", breaks = seq(1970, 2016, 1))

In [ ]:
#unique(df$region_txt)
#unique(df$region_txt)
ggplot(df, aes(region_txt)) +
  geom_bar(fill = "#0073C2FF")+labs(x="regions",y="n0. of attacks")+theme(axis.text.x = element_text(angle = 90, hjust =1, vjust = 0.2))

In [ ]:
ggplot(df, aes(attacktype1_txt)) +
  geom_bar(fill = "#0073C2FF")+labs(x="attack type",y="no. of attacks")+theme(axis.text.x = element_text(angle = 90, hjust =1, vjust = 0.2))

In [ ]:
df %>% filter(nkill > 0) -> dfk
dfk %>% group_by(iyear,region_txt) %>% summarise(nkills = sum(nkill)) %>% ungroup() -> dfyr
colnames(dfyr)<-c("Year","Region","Killed")
ggplot(data = dfyr, aes(x = Year, y = Killed, colour = Region)) +       
   geom_line() + geom_point() + theme_bw()

In [ ]:
counts = table(df$gname)
dict = as.data.frame(counts) %>% arrange(desc(Freq))
#par(mar = c(5,20,5.7,5)) 
#barplot(dict$Freq[2:20], names = dict$Var1[2:20], las = 2, cex.names = 1, horiz = TRUE, xlim = c(0,10000), main = "The Groups causing attacks")
library(ggplot2)
g <- ggplot(dict[2:15,], aes(x=Var1, y=Freq))+geom_bar(stat="identity")+theme(axis.text.x = element_text(angle = 90, hjust = 1))+ggtitle("significant terrorist groups based on their attacks")
g

In [ ]:
country_freq = as.data.frame(table(df$country_txt))
colnames(country_freq) = c("country","frequency")
matched = joinCountryData2Map(country_freq, joinCode="NAME", nameJoinColumn="country")
mapCountryData(matched,nameColumnToPlot="frequency", mapTitle="terrorist attacks distribution based on geography", catMethod = "pretty", colourPalette = "heat")

In [ ]:
line_plot =  as.data.frame(table(df$iyear))
ggplot(data=line_plot, aes(x=Var1, y=Freq, group=1)) +
  geom_line(color="red")+
  geom_point()+theme(axis.text.x = element_text(angle = 90, hjust = 1))+ggtitle("trend of terrorist attacks around the world")

In [ ]:
#trend = df %>% group_by(weaptype1_txt) %>% group_by(iyear) %>% summarise(count = n())
trend = df %>% group_by(iyear,weaptype1_txt) %>% summarise(count = n())
#ggplot(data=trend, aes(x=iyear, y=count, group=weaptype1_txt)) +
 # geom_bar(aes(linetype=weaptype1_txt))+theme(plot.margin = unit(c(1,0,1,1), "cm"))+
  #geom_point()
stacked = ggplot() + geom_bar(aes(y = count, x = iyear, fill = weaptype1_txt), data = trend,
                           stat="identity")+theme(legend.position="bottom", legend.direction="horizontal", legend.title = element_blank())+
            ggtitle("The use of weapons across the year gap 1970-2017")+scale_x_continuous(breaks=seq(1970,2017,1))+theme(axis.text.x = element_text(angle = 90, hjust = 1))
stacked